In [3]:
import pandas as pd
import numpy as np
wm = 1
dm = 1
vm = 1
xls = pd.ExcelFile('Wednesday.xlsx')
df = pd.read_excel(xls, 'Wednesday_Locations (2)')
X=df.loc[:,['TOZIP','CUBE']]
Locations = df.loc[:,['X','Y']]
Order_Quantity = df.loc[:,['CUBE']].to_numpy()
Orders = X.to_numpy()
Points = Locations.to_numpy()
xls = pd.ExcelFile('Wednesday_Distance.xlsx')
df2= pd.read_excel(xls, 'Distance_Matrix')
Distances = df2.to_numpy()
distribution_center = [-71.17555556, 42.54555556]
##WT = [ 30, 30, 30, 30, 30,  30, 53.88, 30, 30, 30,  30, 30, 30, 30, 44.37,  30, 30, 30, 30, 30,  30, 30, 30, 30, 30,  30, 30, 30, 30, 30, 30, 30, 30, 30]
WT = df.loc[:,['Waiting_Time']].to_numpy()*wm
len(Orders)
result = []
max_capacity = 3200
max_time = 600
max_driving_time = 11*60
max_service_time = 14*60
velocity = 0.67*vm

route = []
route_capacity = 0
route_time = 0
prev_index = 0
driving_time = 0
temp_route_time = 0
service_time = 0
serv_prev_index = 0
for order in Orders:
    order_index = ((np.where(Orders == order[0])[0][0]) + 1)
    if not route or len(route)== 1:
        route_time = 0
    else: 
        prev_index = prev_index
        serv_prev_index = serv_prev_index 
    if order_index == 39:
        route.append(order)
        result.append(route)
        break
    else:
        print(prev_index)
        if ((route_capacity + (order[1]*dm)) < max_capacity):
            if ((route_time + int(Distances[prev_index, order_index] / velocity) + WT[order_index-1]*wm) < max_time):
                if (service_time + int(Distances[serv_prev_index, order_index] / velocity) + int(Distances[order_index, 0] / velocity) + WT[order_index-1]*wm < max_service_time):
                    if ((driving_time + int(Distances[serv_prev_index, order_index] / velocity) + int(Distances[order_index, 0] / velocity))  < max_driving_time): 
                        service_time += (int(Distances[serv_prev_index, order_index] / velocity) + WT[order_index-1]*wm)
                        route.append(order)
                        route_capacity += order[1]*dm
                        route_time += (int(Distances[prev_index, order_index] / velocity) + WT[order_index-1]*wm)
                        driving_time += int(Distances[serv_prev_index, order_index] / velocity)
                        prev_index = ((np.where(Orders == order[0])[0][0]) + 1)
                        serv_prev_index = ((np.where(Orders == order[0])[0][0]) + 1)
                        continue
                    else:
                        result.append(route)
                        route = [order]
                        route_capacity = order[1]*dm
                        route_time += WT[order_index-1]*wm
                        driving_time = int(Distances[0, order_index] / velocity)
                        service_time = int(Distances[0, order_index] / velocity + WT[order_index-1]*wm)
                        serv_prev_index = ((np.where(Orders == order[0])[0][0]) + 1)
                        prev_index = ((np.where(Orders == order[0])[0][0]) + 1)
                        print(result)  
                else:
                    result.append(route)
                    route = [order]
                    route_capacity = order[1]*dm
                    route_time += WT[order_index-1]*wm
                    driving_time = int(Distances[0, order_index] / velocity)
                    service_time = int(Distances[0, order_index] / velocity + WT[order_index-1]*wm)
                    serv_prev_index = ((np.where(Orders == order[0])[0][0]) + 1)
                    prev_index = ((np.where(Orders == order[0])[0][0]) + 1)
                    print(result)
            else:
                result.append(route)
                route = [order]
                route_capacity = order[1]*dm
                route_time += WT[order_index-1]*wm
                driving_time = int(Distances[0, order_index] / velocity)
                service_time = int(Distances[0, order_index] / velocity + WT[order_index-1]*wm)
                serv_prev_index = ((np.where(Orders == order[0])[0][0]) + 1)
                prev_index = ((np.where(Orders == order[0])[0][0]) + 1)
                print(result)
        else:
            result.append(route)
            route = [order]
            route_capacity = order[1]*dm
            route_time += WT[order_index-1]*wm
            driving_time = int(Distances[0, order_index] / velocity)
            service_time = int(Distances[0, order_index] / velocity + WT[order_index-1]*wm)
            serv_prev_index = ((np.where(Orders == order[0])[0][0]) + 1)
            prev_index = ((np.where(Orders == order[0])[0][0]) + 1)
            print(result)
            continue
# Print the result
for i, route in enumerate(result):
    print("Route", i + 1)
    print("Order ID\tZipcode\tQuantity")
    for order in route:
        order_index = np.where(Orders == order[0])[0][0]+1
        print(order_index, "\t\t", order[0], "\t", order[1])
    print("")
result_routes = [[order[0] for order in route] for route in result]
print(result_routes)
result_routes_order = [[np.where(Orders == order[0])[0][0]+1 for order in route] for route in result]
print(result_routes_order)
total_distance = 0
route_distnace = []
driving_time = []
const_time = []
for rou in result_routes_order:
    od = 0
    ed = 0
    const = 0
    const_t = 0
    od += int(Distances[0][rou[0]] + Distances[rou[-1]][0])
    for j in range(len(rou)-1): 
        ed += int(Distances[rou[j]][rou[j+1]])
        const += WT[rou[j]]
    const_t = (ed/velocity) + const
    route_distnace.append(ed)
    const_time.append(const_t)
    driving_time.append(ed/velocity)
    total_distance += od + ed

import pandas as pd
import numpy as np
from py2opt.routefinder import RouteFinder
xls = pd.ExcelFile('Wednesday_Distance.xlsx')
df2= pd.read_excel(xls, 'Distance_Matrix')
distance_matrix = df2.to_numpy()
routes = result_routes_order
import numpy as np

# define distance_matrix here or import it from a library
optimized_routes = []
route_capacity = []
total_distance = 0
for route in routes:
    if len(route)>=3:
        cities_names = route

        # create a matrix filled with zeros to store the distances
        dist_mat = np.zeros((len(cities_names), len(cities_names)))

        # fill the matrix with the distances between cities
        for i in range(len(cities_names)):
            for j in range(len(cities_names)):
                dist_mat[i][j] = distance_matrix[cities_names[i]][cities_names[j]]

        # use the distance matrix to find the best route
        route_finder = RouteFinder(dist_mat, cities_names, iterations=10000)
        best_distance, best_route = route_finder.solve()
        total_distance += best_distance
        optimized_routes.append(best_route)
        # print the results
        print(best_distance)
    else: 
        optimized_routes.append(route)
print(optimized_routes)
total_distance = 0
route_distnace = []
driving_time = []
const_time = []
service_time = []
d_time = []
for rou in optimized_routes:
    od = 0
    ed = 0
    const = 0
    const_t = 0
    od += int(distance_matrix[0][rou[0]] + distance_matrix[rou[-1]][0])
    for j in range(len(rou)-1): 
        ed += int(distance_matrix[rou[j]][rou[j+1]])
        const += WT[rou[j]-1]
    for j in range(len(rou)-1):
        if j == len(rou)-2:
            const += WT[rou[j+1]-1]
    const_t = (ed/(velocity)) + const 
    route_distnace.append(ed)
    const_time.append(const_t)
    driving_time.append(ed/(velocity))
    total_distance += od + ed
    d_time.append((od + ed)/(velocity))
    service_time.append((od + ed)/(velocity) + + const)

print("route_distance =", route_distnace)
print("driving_time =", driving_time)
print("Route_time =", const_time)
print("Total Route driving time =", d_time)
print("Total service time =", service_time)
print('\nnumber of routes =')
print(len(route_distnace))
print("total_distance =", total_distance)

for route in optimized_routes:
    cap = 0
    for j in range(len(route)):
        cap += Order_Quantity[route[j]-1]
    route_capacity.append(cap)
print("route_capacity =", route_capacity)

C:\Users\mdsazidr\Anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\mdsazidr\Anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


0
1
2
3
4
5
6
7
[[array([1060,  151], dtype=int64), array([1101,  140], dtype=int64), array([1606,  159], dtype=int64), array([1701,  137], dtype=int64), array([1752,  164], dtype=int64), array([1801,  461], dtype=int64), array([1854,  159], dtype=int64)]]
8
9
[[array([1060,  151], dtype=int64), array([1101,  140], dtype=int64), array([1606,  159], dtype=int64), array([1701,  137], dtype=int64), array([1752,  164], dtype=int64), array([1801,  461], dtype=int64), array([1854,  159], dtype=int64)], [array([1876, 2800], dtype=int64), array([2110,  328], dtype=int64)]]
10
11
12
13
14
15
16
[[array([1060,  151], dtype=int64), array([1101,  140], dtype=int64), array([1606,  159], dtype=int64), array([1701,  137], dtype=int64), array([1752,  164], dtype=int64), array([1801,  461], dtype=int64), array([1854,  159], dtype=int64)], [array([1876, 2800], dtype=int64), array([2110,  328], dtype=int64)], [array([2111,  120], dtype=int64), array([2114,  280], dtype=int64), array([2115,  420], dtype=i

287 msec
287 msec
288 msec
288 msec
288 msec
288 msec
288 msec
288 msec
288 msec
288 msec
288 msec
288 msec
288 msec
289 msec
289 msec
289 msec
289 msec
289 msec
289 msec
289 msec
289 msec
289 msec
289 msec
290 msec
290 msec
290 msec
290 msec
290 msec
290 msec
290 msec
290 msec
290 msec
291 msec
291 msec
291 msec
291 msec
291 msec
291 msec
291 msec
291 msec
291 msec
291 msec
291 msec
292 msec
292 msec
292 msec
292 msec
292 msec
292 msec
292 msec
292 msec
292 msec
292 msec
292 msec
292 msec
292 msec
292 msec
293 msec
293 msec
293 msec
293 msec
293 msec
293 msec
293 msec
293 msec
293 msec
293 msec
293 msec
293 msec
293 msec
293 msec
294 msec
294 msec
294 msec
294 msec
294 msec
294 msec
294 msec
294 msec
294 msec
294 msec
294 msec
294 msec
294 msec
295 msec
295 msec
295 msec
295 msec
295 msec
295 msec
295 msec
295 msec
295 msec
295 msec
295 msec
296 msec
296 msec
296 msec
296 msec
296 msec
296 msec
296 msec
296 msec
296 msec
296 msec
296 msec
297 msec
297 msec
297 msec
297 msec
297 msec
2

525 msec
525 msec
525 msec
525 msec
525 msec
525 msec
525 msec
525 msec
526 msec
526 msec
526 msec
526 msec
526 msec
526 msec
527 msec
527 msec
527 msec
527 msec
527 msec
527 msec
527 msec
527 msec
527 msec
527 msec
527 msec
527 msec
528 msec
528 msec
528 msec
528 msec
528 msec
528 msec
528 msec
528 msec
528 msec
528 msec
528 msec
528 msec
528 msec
528 msec
528 msec
528 msec
529 msec
529 msec
529 msec
529 msec
529 msec
529 msec
529 msec
529 msec
529 msec
529 msec
529 msec
530 msec
530 msec
530 msec
530 msec
530 msec
530 msec
530 msec
530 msec
531 msec
531 msec
531 msec
531 msec
531 msec
531 msec
531 msec
531 msec
531 msec
531 msec
531 msec
531 msec
531 msec
532 msec
532 msec
532 msec
532 msec
532 msec
532 msec
532 msec
532 msec
532 msec
532 msec
532 msec
533 msec
533 msec
533 msec
533 msec
533 msec
533 msec
533 msec
533 msec
533 msec
533 msec
533 msec
533 msec
534 msec
534 msec
534 msec
534 msec
534 msec
534 msec
534 msec
534 msec
534 msec
534 msec
534 msec
534 msec
535 msec
535 msec
5

57 msec
57 msec
58 msec
58 msec
58 msec
58 msec
58 msec
58 msec
58 msec
58 msec
58 msec
59 msec
59 msec
59 msec
59 msec
59 msec
59 msec
59 msec
59 msec
59 msec
59 msec
59 msec
59 msec
60 msec
60 msec
60 msec
60 msec
60 msec
60 msec
60 msec
60 msec
60 msec
60 msec
60 msec
60 msec
61 msec
61 msec
61 msec
61 msec
62 msec
62 msec
62 msec
62 msec
62 msec
62 msec
62 msec
62 msec
62 msec
62 msec
62 msec
62 msec
62 msec
62 msec
62 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
63 msec
64 msec
64 msec
64 msec
64 msec
64 msec
64 msec
64 msec
64 msec
64 msec
64 msec
64 msec
64 msec
64 msec
65 msec
65 msec
65 msec
65 msec
65 msec
65 msec
65 msec
65 msec
65 msec
65 msec
65 msec
66 msec
66 msec
66 msec
66 msec
66 msec
66 msec
66 msec
66 msec
66 msec
66 msec
66 msec
67 msec
67 msec
67 msec
67 msec
67 msec
67 msec
67 msec
67 msec
68 msec
68 msec
68 msec
68 msec
68 msec
68 msec
68 msec
68 msec
68 msec


...

1026 msec
1026 msec
1026 msec
1026 msec
1027 msec
1027 msec
1027 msec
1027 msec
1027 msec
1027 msec
1028 msec
1028 msec
1028 msec
1029 msec
1029 msec
1029 msec
1029 msec
1029 msec
1029 msec
1029 msec
1029 msec
1030 msec
1030 msec
1031 msec
1031 msec
1031 msec
1031 msec
1031 msec
1031 msec
1031 msec
1031 msec
1032 msec
1032 msec
1032 msec
1032 msec
1032 msec
1032 msec
1032 msec
1032 msec
1033 msec
1033 msec
1033 msec
1033 msec
1033 msec
1033 msec
1033 msec
1034 msec
1034 msec
1034 msec
1034 msec
1034 msec
1034 msec
1034 msec
1035 msec
1035 msec
1035 msec
1035 msec
1036 msec
1036 msec
1036 msec
1036 msec
1036 msec
1036 msec
1036 msec
1036 msec
1037 msec
1037 msec
1037 msec
1037 msec
1037 msec
1037 msec
1037 msec
1037 msec
1037 msec
1038 msec
1038 msec
1038 msec
1038 msec
1038 msec
1038 msec
1038 msec
1038 msec
1039 msec
1039 msec
1039 msec
1039 msec
1039 msec
1040 msec
1040 msec
1040 msec
1040 msec
1040 msec
1040 msec
1040 msec
1040 msec
1041 msec
1041 msec
1041 msec
1041 msec
1041 msec


105 msec
105 msec
106 msec
106 msec
106 msec
106 msec
106 msec
106 msec
106 msec
107 msec
107 msec
107 msec
107 msec
107 msec
107 msec
108 msec
108 msec
108 msec
108 msec
108 msec
109 msec
109 msec
109 msec
109 msec
109 msec
109 msec
109 msec
110 msec
110 msec
110 msec
110 msec
110 msec
110 msec
110 msec
110 msec
110 msec
110 msec
110 msec
110 msec
110 msec
111 msec
111 msec
111 msec
111 msec
111 msec
111 msec
111 msec
111 msec
111 msec
111 msec
111 msec
111 msec
111 msec
111 msec
111 msec
112 msec
112 msec
112 msec
112 msec
112 msec
112 msec
112 msec
112 msec
112 msec
112 msec
112 msec
113 msec
113 msec
113 msec
113 msec
113 msec
113 msec
113 msec
113 msec
114 msec
114 msec
114 msec
114 msec
114 msec
114 msec
114 msec
114 msec
114 msec
114 msec
114 msec
115 msec
115 msec
115 msec
115 msec
115 msec
115 msec
116 msec
116 msec
116 msec
116 msec
116 msec
116 msec
116 msec
116 msec
116 msec
116 msec
116 msec
116 msec
116 msec
116 msec
116 msec
117 msec
117 msec
117 msec
117 msec
117 msec
1

324 msec
324 msec
324 msec
324 msec
324 msec
324 msec
324 msec
324 msec
324 msec
325 msec
325 msec
325 msec
325 msec
325 msec
325 msec
325 msec
326 msec
326 msec
326 msec
326 msec
326 msec
326 msec
326 msec
327 msec
327 msec
327 msec
327 msec
327 msec
327 msec
327 msec
327 msec
328 msec
328 msec
328 msec
328 msec
328 msec
328 msec
328 msec
328 msec
328 msec
329 msec
329 msec
329 msec
329 msec
329 msec
329 msec
329 msec
330 msec
330 msec
330 msec
330 msec
330 msec
330 msec
330 msec
330 msec
330 msec
330 msec
330 msec
330 msec
330 msec
331 msec
331 msec
331 msec
331 msec
331 msec
331 msec
331 msec
331 msec
331 msec
331 msec
331 msec
331 msec
332 msec
332 msec
332 msec
332 msec
332 msec
332 msec
332 msec
332 msec
332 msec
332 msec
332 msec
332 msec
333 msec
333 msec
333 msec
333 msec
333 msec
333 msec
333 msec
333 msec
333 msec
334 msec
334 msec
334 msec
334 msec
334 msec
334 msec
334 msec
334 msec
334 msec
335 msec
335 msec
335 msec
335 msec
335 msec
335 msec
335 msec
335 msec
336 msec
3

541 msec
542 msec
542 msec
542 msec
542 msec
542 msec
542 msec
542 msec
542 msec
542 msec
543 msec
543 msec
543 msec
543 msec
543 msec
543 msec
543 msec
543 msec
543 msec
543 msec
543 msec
544 msec
544 msec
544 msec
544 msec
544 msec
544 msec
544 msec
544 msec
544 msec
544 msec
544 msec
545 msec
545 msec
545 msec
545 msec
545 msec
545 msec
545 msec
545 msec
545 msec
545 msec
545 msec
545 msec
546 msec
546 msec
546 msec
546 msec
546 msec
546 msec
546 msec
546 msec
546 msec
546 msec
546 msec
547 msec
547 msec
547 msec
547 msec
547 msec
547 msec
547 msec
547 msec
547 msec
547 msec
547 msec
548 msec
548 msec
548 msec
548 msec
548 msec
548 msec
548 msec
548 msec
548 msec
548 msec
548 msec
548 msec
548 msec
549 msec
549 msec
549 msec
549 msec
549 msec
549 msec
549 msec
549 msec
549 msec
549 msec
550 msec
550 msec
550 msec
550 msec
550 msec
550 msec
550 msec
550 msec
550 msec
550 msec
550 msec
550 msec
551 msec
551 msec
551 msec
551 msec
551 msec
551 msec
551 msec
551 msec
551 msec
551 msec
5

1028 msec
1029 msec
1029 msec
1029 msec
1029 msec
1029 msec
1029 msec
1029 msec
1030 msec
1030 msec
1030 msec
1030 msec
1030 msec
1030 msec
1031 msec
1031 msec
1031 msec
1031 msec
1031 msec
1031 msec
1031 msec
1032 msec
1032 msec
1032 msec
1032 msec
1032 msec
1032 msec
1032 msec
1033 msec
1033 msec
1033 msec
1033 msec
1033 msec
1033 msec
1034 msec
1034 msec
1034 msec
1034 msec
1034 msec
1034 msec
1034 msec
1034 msec
1034 msec
1034 msec
1034 msec
1035 msec
1035 msec
1035 msec
1035 msec
1035 msec
1035 msec
1035 msec
1035 msec
1035 msec
1035 msec
1035 msec
1035 msec
1036 msec
1036 msec
1036 msec
1036 msec
1036 msec
1036 msec
1036 msec
1036 msec
1036 msec
1036 msec
1036 msec
1037 msec
1037 msec
1037 msec
1037 msec
1037 msec
1037 msec
1037 msec
1037 msec
1037 msec
1037 msec
1038 msec
1038 msec
1038 msec
1038 msec
1038 msec
1038 msec
1038 msec
1038 msec
1038 msec
1038 msec
1038 msec
1039 msec
1039 msec
1039 msec
1039 msec
1039 msec
1039 msec
1039 msec
1039 msec
1039 msec
1039 msec
1039 msec
